In [1]:
import os

In [2]:
%pwd

'f:\\Complete-ML-Project-with-MLFlow-\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'f:\\Complete-ML-Project-with-MLFlow-'

In [5]:
from dataclasses import dataclass
from pathlib import Path

# Entity >> Config.yaml
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path 

In [6]:
from MLProject.constants import * # CONFIG_FILE_PATH,PARAMS_FILE_PATH,SCHEMA_FILE_PATH
from MLProject.utils.common import read_yaml, create_directories

In [7]:
# Configuration manager
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
# Download the dataset and store it 
import os
import urllib.request as request
import zipfile
from MLProject import logger
from MLProject.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
# Pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-08-08 08:13:02,854: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-08 08:13:02,861: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-08 08:13:02,866: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-08 08:13:02,868: INFO: common: created directory at: artifacts]


[2023-08-08 08:13:02,871: INFO: common: created directory at: artifacts/data_ingestion]
[2023-08-08 08:13:05,180: INFO: 2280763589: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8EDE:06DB:9000F:AD9B5:64D1ABB5
Accept-Ranges: bytes
Date: Tue, 08 Aug 2023 02:43:04 GMT
Via: 1.1 varnish
X-Served-By: cache-qpg1221-QPG
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1691462584.728184,VS0,VE361
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: e6905fe0c6d7c0e87a7b92d09944809cdd7062c7
Expires: Tue, 08 A